In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
import os
import time
import sys
import numpy as np
import torch
sys.path.append('/home/chaofan/powerknowledge/data')
# sys.path.append('data/')
from read_PLAID_data import read_processed_data,get_feature_name

def one_hot(labels,class_list):
    labels_num=[]
    for label in labels:
        labels_num.append(class_list.index(label))
    labels_num=torch.from_numpy(np.array(labels_num))
    batch_size=len(labels)
    class_num=len(class_list)
    labels_num=labels_num.resize_(batch_size,1)
    m_zeros=torch.zeros(batch_size,class_num)
    onehot=m_zeros.scatter_(1,labels_num,1)
    return onehot.numpy()

def to_num(labels,class_list):
    labels_num=[]
    for label in labels:
        labels_num.append(class_list.index(label))
    return np.array(labels_num)

In [2]:
# test normal appliance
start_reading_time = time.time()
feature_select=get_feature_name('/home/chaofan/powerknowledge/data/source/submetered_zengj/total')
selected_label = [
    'Air Conditioner', 'Coffee maker', 'Fan', 'Fridge', 'Hair Iron',
    'Hairdryer', 'Heater', 'Incandescent Light Bulb', 'Microwave',
    'Soldering Iron', 'Vacuum', 'Washing Machine', 'Water kettle'
]
x_normal_train, y_normal_train, index_normal_train = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/training')

x_normal_validation, y_normal_validation, index_normal_validation = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/validation')

x_normal_trainval = np.concatenate((x_normal_train, x_normal_validation), axis=0)
y_normal_trainval = np.concatenate((y_normal_train, y_normal_validation), axis=0)

x_normal_test, y_normal_test, index_normal_test = read_processed_data(
    'type',
    type_header='appliance',
    selected_label=selected_label,
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/testing')

y_normal_train_onehot=one_hot(y_normal_train,selected_label)
y_normal_validation_onehot=one_hot(y_normal_validation,selected_label)
y_normal_trainval_onehot=one_hot(y_normal_trainval,selected_label)
y_normal_test_onehot=one_hot(y_normal_test,selected_label)

y_normal_train_num=to_num(y_normal_train,selected_label)
y_normal_validation_num=to_num(y_normal_validation,selected_label)
y_normal_trainval_num=to_num(y_normal_trainval,selected_label)
y_normal_test_num=to_num(y_normal_test,selected_label)

print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

normal_rf = RandomForestClassifier(n_estimators=200,
                                min_samples_split=10,
                                min_samples_leaf=5,
                                max_depth=8,
                                max_features='auto',
                                random_state=10)
normal_rf.fit(x_normal_trainval, y_normal_trainval)

y_normal_trainval_pred = normal_rf.predict(x_normal_trainval)
y_normal_trainval_predprob = normal_rf.predict_proba(x_normal_trainval)

y_normal_test_pred = normal_rf.predict(x_normal_test)
y_normal_test_predprob = normal_rf.predict_proba(x_normal_test)

print("Accuracy : %.4g" % metrics.accuracy_score(y_normal_trainval, y_normal_trainval_pred))
print("Accuracy : %.4g" % metrics.accuracy_score(y_normal_test, y_normal_test_pred))

print("AUC Score (test): %f" %
      metrics.roc_auc_score(y_normal_test_onehot, y_normal_test_predprob, average='micro'))

y_normal_trainval_predprob=torch.from_numpy(y_normal_trainval_predprob)


finished loading data, cost 5.133s
Accuracy : 0.9889
Accuracy : 0.7995
AUC Score (test): 0.981166


In [5]:
import torch.nn as nn

class nn_student(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim):
        super(nn_student,self).__init__()
        self.fc=nn.Sequential(
            nn.Linear(input_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim,hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim,hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim,hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim,hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim,hidden_dim),
            nn.Sigmoid(),
            nn.Linear(hidden_dim,output_dim),
        )
    def forward(self,x):
        return self.fc(x)

x_normal_trainval_tensor=torch.from_numpy(x_normal_trainval).to(torch.float32)
y_normal_trainval_onehot_tensor=torch.from_numpy(y_normal_trainval_onehot).to(torch.float32)
y_normal_trainval_tensor=torch.from_numpy(y_normal_trainval_num)

x_normal_test_tensor=torch.from_numpy(x_normal_test).to(torch.float32)
y_normal_test_onehot_tensor=torch.from_numpy(y_normal_test_onehot).to(torch.float32)

for times in range(1,10,1):
    nn_clf=nn_student(34,32,14)
    optimizer=torch.optim.SGD(nn_clf.parameters(),lr=0.1)
    loss_fn=torch.nn.CrossEntropyLoss()
    for i in range(10000):
        acc_train=0.0
        acc_test=0.0
        prediction=nn_clf(x_normal_trainval_tensor)
        loss=loss_fn(prediction,y_normal_trainval_tensor)  # crossentroy 对于多分类直接用数字
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i%9999==0 and i!=0:
            acc_train += np.sum(np.argmax(prediction.cpu().data.numpy(), axis=1) ==
                    np.argmax(y_normal_trainval_onehot,axis=1))
            x_normal_test_pred = nn_clf(x_normal_test_tensor)
            acc_test += np.sum(np.argmax(x_normal_test_pred.cpu().data.numpy(), axis=1) ==
                            np.argmax(y_normal_test_onehot,axis=1))
            print('acc_train:%.4f,loss:%.4f,acc_test:%.4f'%(acc_train/len(y_normal_trainval_onehot),loss.item(),acc_test/len(y_normal_test_onehot)))
    # test_pred=nn_clf(x_test_tensor)

acc_train:0.9899,loss:0.0605,acc_test:0.6900
acc_train:0.9717,loss:0.1001,acc_test:0.6224
acc_train:0.9575,loss:0.1124,acc_test:0.5664
acc_train:0.9757,loss:0.0967,acc_test:0.6573
acc_train:0.9717,loss:0.1162,acc_test:0.6946
acc_train:0.9706,loss:0.1049,acc_test:0.6084
acc_train:0.9949,loss:0.0548,acc_test:0.6597
acc_train:0.9848,loss:0.0899,acc_test:0.6294
acc_train:0.9464,loss:0.1686,acc_test:0.5828


In [4]:
# 找最佳T和a
import torch.nn as nn
import torch.nn.functional as F

class nn_student(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim):
        super(nn_student,self).__init__()
        self.fc=nn.Sequential(
            nn.Linear(input_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,hidden_dim),
            nn.Sigmoid(),
            nn.Linear(hidden_dim,output_dim),
        )
    def forward(self,x):
        return self.fc(x)

x_normal_trainval_tensor=torch.from_numpy(x_normal_trainval).to(torch.float32)
y_normal_trainval_onehot_tensor=torch.from_numpy(y_normal_trainval_onehot).to(torch.float32)
y_normal_trainval_tensor=torch.from_numpy(y_normal_trainval_num)


# alpha=0.8
# T=2
best_score=0.0
best_alpha=0
best_T=0
for alpha in [0.05,0.1,0.2,0.25,0.5,0.75,0.8,0.9,0.95,1]:
    for T in [1,2,3,4,5,6,7,8,9,10]:
        nn_clf=nn_student(34,32,13)
        optimizer=torch.optim.SGD(nn_clf.parameters(),lr=0.01)
        loss_fn1=torch.nn.CrossEntropyLoss()
        loss_fn2=torch.nn.KLDivLoss()
        for i in range(100000):
            acc=0.0
            prediction=nn_clf(x_normal_trainval_tensor)
            loss1=loss_fn1(prediction,y_normal_trainval_tensor)
            
            output_student=F.log_softmax(prediction/T,dim=1).to(torch.float32)
            # output_teacher=F.softmax(y_trainval_predprob/T,dim=1).to(torch.float32)
            output_teacher=y_normal_trainval_predprob.to(torch.float32)
            loss2=loss_fn2(output_student,output_teacher)*T*T
            optimizer.zero_grad()
            loss=loss1*(1-alpha)+loss2*alpha
            loss.backward()
            optimizer.step()
            # if i%1000==0:
            #     acc += np.sum(np.argmax(prediction.cpu().data.numpy(), axis=1) ==
            #             np.argmax(y_trainval_onehot,axis=1))
            #     print('acc:%.5f'%(acc/len(y_trainval_onehot)))
            #     print('[%d, %5d] loss: %.4f loss1: %.4f loss2: %.4f' %(i, 100000, loss.item(), loss1.item(), loss2.item()))

        x_normal_test_tensor=torch.from_numpy(x_normal_test).to(torch.float32)
        y_normal_test_onehot_tensor=torch.from_numpy(y_normal_test_onehot).to(torch.float32)
        test_pred=nn_clf(x_normal_test_tensor)
        acc=0.0
        acc += np.sum(np.argmax(test_pred.cpu().data.numpy(), axis=1) ==
                        np.argmax(y_normal_test_onehot,axis=1))
        acc=acc/len(y_normal_test_onehot)
        if acc>best_score:
            best_score=acc
            best_alpha=alpha
            best_T=T
        print('alpha:%.2f,T:%02d,acc:%.5f'%(alpha,T,acc))

KeyboardInterrupt: 

In [7]:
# 查看最佳参数下收敛情况
import torch.nn as nn
import torch.nn.functional as F

class nn_student(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim):
        super(nn_student,self).__init__()
        self.fc=nn.Sequential(
            nn.Linear(input_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,hidden_dim),
            nn.Sigmoid(),
            nn.Linear(hidden_dim,output_dim),
        )
    def forward(self,x):
        return self.fc(x)

x_trainval_tensor=torch.from_numpy(x_trainval).to(torch.float32)
y_trainval_onehot_tensor=torch.from_numpy(y_trainval_onehot).to(torch.float32)
y_trainval_tensor=torch.from_numpy(y_trainval_num)


alpha=0.9
T=3

nn_clf=nn_student(34,32,13)
optimizer=torch.optim.SGD(nn_clf.parameters(),lr=0.01)
loss_fn1=torch.nn.CrossEntropyLoss()
loss_fn2=torch.nn.KLDivLoss()
for i in range(100000):
    acc=0.0
    prediction=nn_clf(x_trainval_tensor)
    loss1=loss_fn1(prediction,y_trainval_tensor)
    
    output_student=F.log_softmax(prediction/T,dim=1).to(torch.float32)
    # output_teacher=F.softmax(y_trainval_predprob/T,dim=1).to(torch.float32)
    output_teacher=y_trainval_predprob.to(torch.float32)
    loss2=loss_fn2(output_student,output_teacher)*T*T
    optimizer.zero_grad()
    loss=loss1*(1-alpha)+loss2*alpha
    loss.backward()
    optimizer.step()
    if i%1000==0:
        acc += np.sum(np.argmax(prediction.cpu().data.numpy(), axis=1) ==
                np.argmax(y_trainval_onehot,axis=1))
        print('acc:%.5f'%(acc/len(y_trainval_onehot)))
        print('[%d, %5d] loss: %.4f loss1: %.4f loss2: %.4f' %(i, 100000, loss.item(), loss1.item(), loss2.item()))

x_test_tensor=torch.from_numpy(x_test).to(torch.float32)
y_test_onehot_tensor=torch.from_numpy(y_test_onehot).to(torch.float32)
test_pred=nn_clf(x_test_tensor)
acc=0.0
acc += np.sum(np.argmax(test_pred.cpu().data.numpy(), axis=1) ==
                np.argmax(y_test_onehot,axis=1))
acc=acc/len(y_test_onehot)

print('alpha:%.2f,T:%02d,acc:%.5f'%(alpha,T,acc))

acc:0.10732
[0, 100000] loss: 1.4579 loss1: 2.7084 loss2: 1.3189
acc:0.45135
[1000, 100000] loss: 1.0761 loss1: 1.7544 loss2: 1.0008
acc:0.49549
[2000, 100000] loss: 0.9243 loss1: 1.5529 loss2: 0.8544
acc:0.53360
[3000, 100000] loss: 0.8311 loss1: 1.4437 loss2: 0.7630
acc:0.54463
[4000, 100000] loss: 0.7625 loss1: 1.3570 loss2: 0.6965
acc:0.55567
[5000, 100000] loss: 0.7091 loss1: 1.2916 loss2: 0.6443
acc:0.58576
[6000, 100000] loss: 0.6662 loss1: 1.2404 loss2: 0.6024
acc:0.59378
[7000, 100000] loss: 0.6315 loss1: 1.1996 loss2: 0.5684
acc:0.61384
[8000, 100000] loss: 0.6028 loss1: 1.1694 loss2: 0.5399
acc:0.64193
[9000, 100000] loss: 0.5789 loss1: 1.1431 loss2: 0.5163
acc:0.65697
[10000, 100000] loss: 0.5573 loss1: 1.1094 loss2: 0.4960
acc:0.67101
[11000, 100000] loss: 0.5378 loss1: 1.0843 loss2: 0.4771
acc:0.67904
[12000, 100000] loss: 0.5184 loss1: 1.0519 loss2: 0.4591
acc:0.68205
[13000, 100000] loss: 0.4990 loss1: 1.0122 loss2: 0.4420
acc:0.70010
[14000, 100000] loss: 0.4816 loss1:

In [3]:
# total data 
from sklearn.model_selection import train_test_split

start_reading_time = time.time()
feature_select=get_feature_name('/home/chaofan/powerknowledge/data/source/submetered_zengj/total')

x, y, index = read_processed_data(
    'type',
    type_header='appliance',
    direaction=1,
    offset=0,
    each_lenth=10,
    feature_select=feature_select,
    source='submetered_zengj/total')

print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))


finished loading data, cost 21.120s
